# 12d - AgriPV: Designing for adecuate crop shading 

This journal supports the process of designing a solar panel configuration to appropriately represent ideal shading conditions for coffee production underneath elevated solar panels. 

The coffee trees would be under and/or in between elevated solar panels (panels would be elevated 6, 8, or 10 ft tall). The light/shade analysis helps determine appropriate panel heights and spacings t0 achieve appropriate shading.  The desired level of shading is maximum of 30% (i.e., 70% of normal, unshaded light). 

Details:
*  The coffee plants are expected to be \~5 ft tall. (5-6 ft tall and 3 ft wide (<a href="https://realgoodcoffeeco.com/blogs/realgoodblog/how-to-grow-a-coffee-plant-at-home#:~:text=However%2C%20you%20must%20keep%20in,tall%20and%203%20feet%20wide">Reference</a>)
*	Location: 18.202142, -66.759187; (18°12'07.7"N 66°45'33.1"W)
*	Desired area of initial analysis: 400-600 ft2 (37-55 m2)
*	Racking: Fixed-tilt panels
*	Panel size: 3.3 feet x 5.4 feet                                    (1m x 1.64m)
*	Analysis variations:
<ul> <li> a.	Panel height: would like to examine heights of 6 ft, 8 ft, and 10 ft hub height. 
<li> b.	Panel spacing (N/W): would like to look at multiple distances (e.g., 2 ft, 3 ft, 4 ft) </li> 
<li> c.	Inter-Row spacing (E/W): would like to look at multiple distances (e.g., 2 ft, 3 ft, 4 ft)! </li> 


Steps on this Journal:
<ol>
    <li> <a href='#step1'> <u><b>Loop to Raytrace and sample irradiance at where Three would be located </u></b></li>
    <li> <a href='#step2'> Calculate GHI for Comparisons </li>
        <ul><li> <a href='#step2a'> Option 1: Raytrace of Empty Field  </li></ul>
        <ul><li> <a href='#step2b'> Option 2: Weather File </li></ul>
    <li> <a href='#step3'> Compile Results </li>
    <li> <a href='#step4'> Plot Results</li>
    <li> <a href='#step5'> <u><b> Raytrace with Tree Geometry <u></b></li>
        <ul><li> <a href='#step5a'>Tree Parameters</li></ul>
        <ul><li> <a href='#step5b'>Loop to Raytrace and Sample Irradiance at Each side of the Tree (N, S, E, W)</li></ul>
        <ul><li> <a href='#step5c'>Single simulation until MakeOct for Getting a PRETTY IMAGE </li></ul>
    <li> <a href='#step6'> Compile Results</li>
    <li> <a href='#step7'>  Plot </li>

</ol>
        

        
![AgriPV Coffee Trees Simulation](../images_wiki/AdvancedJournals/AgriPV_CoffeeTrees.PNG)

 
While we have HPC scripts to do the below simulation, this journals runs all of the above so it might take some time, as there are 109 combinations of parameters explored

In [1]:
import bifacial_radiance
import os
from pathlib import Path
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
testfolder = str(Path().resolve().parent.parent / 'bifacial_radiance' / 'TEMP' /  'Tuscon_test')
if not os.path.exists(testfolder):
    os.makedirs(testfolder)
    
resultsfolder = os.path.join(testfolder, 'results')

### General Parameters and Variables

In [3]:
lat = 32.25
lon = -110.9  #Lat and Long for Nespelem, WA

albedo = 0.31 # Average Albedo from Nespelem
              # International Multi-Conference for Engineering, Education, and Technology
              # Q is this the correct Albedo? There was nothing in the Powerpoint

ft2m = 0.3048
    
# Loops
azimuths = np.array([180, 150, 210]) #cycling through 3 different azimuths
azimuth_strings = ['180','150','210']


# Constants
clearance_height = 1.45 # Meters
xgap =  3 * ft2m
ygap = .02 # Meters
# D (meters)is a variable that represents the spacing between rows, not-considering the collector areas.
D = 2.8    
tilt = 25
x = 1
y = 2     
nMods = 7
nRows = 2
numpanels = 4
pitch = ((y * numpanels) * np.cos(np.radians(tilt)))+D
punder=((y * numpanels) * np.cos(np.radians(tilt)))
moduletype = 'test-module' #Need to make sure this is right module
hpc = False
sim_general_name = 'nespelem'
print(pitch, punder)

#width = nMods * x +(nMods-1)*(xgap)+2
#xinc = width / 14
#xstart = -width/2
#length = (4*y+(3*ygap))* np.cos(np.radians(tilt))*2+D+2
#ystart = length/1.328
#yinc = -length/9

#print(width,length,xstart,xinc,ystart,yinc)

6.454232997331772 3.6542329973317726


In [ ]:
#if not os.path.exists(os.path.join(testfolder, 'EPWs')):
    #demo = bifacial_radiance.RadianceObj('test',testfolder)  
    #epwfile = demo.getEPW(lat,lon)    
#else:
   #epwfile = r'EPWs\USA_WA_Ephrata.Muni.AP.727826_TMY3.epw' 
    #Not nespelem but close. How should I use the Nespelem weather file

In [6]:
epwfile = r'EPWs\nespelem_wa_48.167039_-118.976656_psm3-tmy_60_tmy_TMY3formatV2.csv' 

<a id='step1'></a>

## 1. Create Geometry using gendaylit and view

In [10]:
demo = bifacial_radiance.RadianceObj(sim_general_name,str(testfolder))  
demo.setGround(albedo)
metdata = demo.readWeatherFile(epwfile, coerce_year=2020) # Name and path

path = C:\Users\jjones\Documents\GitHub\bifacial_radiance\bifacial_radiance\TEMP\Nespelem_test2
Loading albedo, 1 value(s), 0.310 avg
1 nonzero albedo values.
8760 line in WeatherFile. Assuming this is a standard hourly WeatherFile for the year for purposes of saving Gencumulativesky temporary weather files in EPW folder.
Coercing year to 2020
Saving file EPWs\metdata_temp.csv, # points: 8760
Calculating Sun position for Metdata that is right-labeled  with a delta of -30 mins. i.e. 12 is 11:30 sunpos


In [ ]:
timestamp = metdata.datetime.index(pd.to_datetime('2020-07-01 16:30:0 -8'))
demo.gendaylit(timestamp)  # Use this to simulate only one hour at a time.
azimuth_test = 180
pitch = ((y * numpanels) * np.cos(np.radians(tilt)))+D
sim_name = sim_general_name+'_az_'+str(round(azimuth,1))
module = demo.makeModule(name=moduletype, x=x, y=y,numpanels = numpanels, xgap=xgap, ygap=ygap)
sceneDict = {'tilt':tilt,'pitch':pitch,'clearance_height':clearance_height,'azimuth':azimuth_test,'nMods': nMods,'nRows': nRows}
scene = demo.makeScene(moduletype=moduletype,sceneDict=sceneDict, radname = sim_name)
octfile = demo.makeOct(demo.getfilelist()) 

In [ ]:
#!rvu -vf views\front.vp -e .01 -pe 0.4 -vp 3.5 -20 22 nespelem.oct

In [ ]:
## Recreate weather and geometry using genCumsky

In [ ]:
# star date and end date for example for monthly data May 1 to May 31
demo = bifacial_radiance.RadianceObj(sim_general_name,str(testfolder))  
demo.setGround(albedo)
demo.readWeatherFile(epwfile) # Name and path
demo.genCumSky()

In [ ]:
# Prepare to run for loop of start and end times for Months This all works. See next cell
#starttimes =['2012-01-01 1:30:0', '2011-02-01 1:30:0', '2003-03-01 1:30:0', '2005-04-01 1:30:0','2005-05-01 1:30:0', '2007-06-01 1:30:0', '2020-07-01 1:30:0', '2020-08-01 1:30:0', '2002-09-01 1:30:0', '1998-10-01 1:30:0', '2007-11-01 1:30:0','2006-12-01 1:30:0' ]
#endtimes = ['2012-01-31 23:30:0', '2011-02-28 23:30:0', '2003-03-31 23:30:0','2005-04-30 23:30:0','2005-05-31 23:30:0', '2007-06-30 23:30:0','2020-07-31 23:30:0', '2020-08-31 23:30:0', '2002-09-30 23:30:0', '1998-10-31 23:30:0', '2007-11-30 23:30:0','2006-12-31 23:30:0'  ]
#demo = bifacial_radiance.RadianceObj(sim_general_name,str(testfolder))  
#demo.setGround(albedo)
#for ti in range (0, len(starttimes)):
#    starttime = pd.to_datetime(starttimes[ti])
#    endtime = pd.to_datetime(endtimes[ti])
#    print(starttime, endtime)
#    demo.readWeatherFile(epwfile, starttime=starttime, endtime=endtime) # Name and path
    #demo.genCumSky()  

## Main part of code. Creates Result files and heatmaps for each month and azimuth

In [8]:
# Parameters for scene size and sensor starts
sensorsx_fb = [1,1]
sensorsy_fb = [4,6*numpanels]
sensorsx_gb = [217,1] #217 for 10cm increase, originally 15
sensorsy_gb = [122,1] #122 for 10cm increase, originally 10
width = nMods * x +(nMods-1)*(xgap)+2
xinc = width / (sensorsx_gb[0] -1) 
xstart = -width/2
length = (4*y+(3*ygap))* np.cos(np.radians(tilt))*2+D+2
#ystart = -length/4
ystart = length/1.328
#yinc = length/9
yinc = -length/(sensorsy_gb[0]-1) 

print(width,length,xstart,xinc,ystart,yinc)

21.703400000000002 12.217222929107942 -10.851700000000001 0.10047870370370371 9.199716061075257 -0.10096878453808217


In [9]:
# Main part of code. I'll test things out below then bring them up here and comment them out
module = demo.makeModule(name=moduletype, x=x, y=y, xgap=xgap, ygap=ygap)
# Prepare to run for loop of start and end times for May to October
starttimes =['2005-05-01 1:30:0', '2007-06-01 1:30:0', '2020-07-01 1:30:0', '2020-08-01 1:30:0', '2002-09-01 1:30:0', '1998-10-01 1:30:0']
endtimes = ['2005-05-31 23:30:0', '2007-06-30 23:30:0','2020-07-31 23:30:0', '2020-08-31 23:30:0', '2002-09-30 23:30:0', '1998-10-31 23:30:0']
months = ['May','Jun','Jul','Aug','Sep','Oct']
demo = bifacial_radiance.RadianceObj(sim_general_name,str(testfolder))  
demo.setGround(albedo)
for ti in range (0, len(starttimes)):
    starttime = pd.to_datetime(starttimes[ti])
    endtime = pd.to_datetime(endtimes[ti])
    month = months[ti]
    demo.readWeatherFile(epwfile, starttime=starttime, endtime=endtime) # Name and path
    demo.genCumSky()
    #Generate sky for each month and run emptyscan for each month
    sim_name = sim_general_name+'_'+month
    sceneDict = {'tilt':0,'pitch':2,'clearance_height':0.005,'azimuth':180, 'nMods': 1, 'nRows': 1} 
    scene = demo.makeScene(moduletype=moduletype,sceneDict=sceneDict, radname = sim_name)
    octfile = demo.makeOct(octname = demo.basename)  
    analysis = bifacial_radiance.AnalysisObj(octfile=octfile, name=sim_name)
    frontscan, backscan = analysis.moduleAnalysis(scene=scene,sensorsx = sensorsx_gb,sensorsy=sensorsy_gb)
    emptyscan = frontscan.copy() 
    emptyscan['xstart'] = xstart
    emptyscan['xinc'] = 0
    emptyscan['sx_xinc'] = xinc
    emptyscan['ystart'] = ystart
    emptyscan['yinc'] = yinc
    emptyscan['zstart'] = 0.05
    emptyscan['zinc'] = 0
    emptyscan['orient'] = '0 0 -1'
    emptybackscan = emptyscan.copy()
    emptybackscan['orient'] = '0 0 1'
    analysis.analysis(octfile, name=sim_name+'_EMPTYSCAN', frontscan=emptyscan, backscan=emptybackscan)

    for az in range (0, len(azimuths)):
        #Build scene and run raytracing for all 3 azimuths
        azimuth = azimuths[az]
        sim_name = sim_general_name+'_'+month+'_az_'+str(round(azimuth,1))
        sceneDict = {'tilt':tilt,'pitch':pitch,'clearance_height':clearance_height,'azimuth':azimuth,'nMods': nMods,'nRows': nRows}
        if azimuth == 180:
            scene = demo.makeScene(moduletype=moduletype,sceneDict=sceneDict, radname = sim_name)
            octfile = demo.makeOct(demo.getfilelist())  
            analysis = bifacial_radiance.AnalysisObj(octfile=octfile, name=sim_name)
            frontscan, backscan = analysis.moduleAnalysis(scene=scene,sensorsx = sensorsx_gb,sensorsy=sensorsy_gb)
            groundscan = frontscan.copy() 
            groundscan['xstart'] = xstart
            groundscan['xinc'] = 0
            groundscan['sx_xinc'] = xinc
            groundscan['ystart'] = ystart
            groundscan['yinc'] = yinc
            groundscan['zstart'] = 0.05
            groundscan['zinc'] = 0
            groundscan['orient'] = '0 0 -1'
        sceneDict['azimuth'] = azimuth
        scene = demo.makeScene(moduletype=moduletype,sceneDict=sceneDict, radname = sim_name)
        octfile = demo.makeOct(demo.getfilelist())
        frontscan, backscan = analysis.moduleAnalysis(scene=scene, sensorsy=sensorsy_fb, sensorsx = sensorsx_fb)
        analysis.analysis(octfile, name=sim_name+'_Module_Analysis', frontscan=frontscan, backscan=backscan)
        analysis.analysis(octfile, name=sim_name+'_Ground_Analysis', frontscan=groundscan, backscan=backscan)
        #Read each azumith file for Wm2 Front
        azimuth_s = azimuth_strings[az]
        file = os.path.join(resultsfolder, 'irr_nespelem_'+month+'_az_'+azimuth_s+'_Ground_Analysis_Front.csv')
        df2 = pd.read_csv(file)
        Wm2 = np.asarray(df2['Wm2Front']) 
        Wms2 = np.reshape(np.ravel(Wm2, order='C'), (122,217), order='F')
        #Read each empty file for Wm2Front
        empty_scan = os.path.join(resultsfolder, 'irr_nespelem_'+month+'_EMPTYSCAN.csv')
        de = pd.read_csv(empty_scan)
        Wmempty = np.asarray(de['Wm2Front']) 
        Wms_empty = np.reshape(np.ravel(Wmempty, order='C'), (122,217), order='F')
        #Calculate Irradiance Facor and plot and save heat map of each month_azumith
        plt.figure(1)
        shading = Wms2/Wms_empty
        fig = plt.imshow(shading, cmap='hot', vmin=np.min(shading).min(), vmax=np.max(shading).max(), interpolation='none', aspect = 'equal')
        plt.colorbar(label='Irr Factor')
        plt.title(month+'_Bifacial Irr Factor (az='+azimuth_s+'), in matrix form')
        fig.axes.get_yaxis().set_visible(False)
        fig.axes.get_xaxis().set_visible(False)
        plt.savefig(month+'_Bifacial_az_'+azimuth_s+'.png')
        plt.clf()
    
        
        






Module Name: test-module
Module test-module updated in module.json
Pre-existing .rad file objects\test-module.rad will be overwritten

path = C:\Users\jjones\Documents\GitHub\bifacial_radiance\bifacial_radiance\TEMP\Nespelem_test2
Loading albedo, 1 value(s), 0.310 avg
1 nonzero albedo values.
8760 line in WeatherFile. Assuming this is a standard hourly WeatherFile for the year for purposes of saving Gencumulativesky temporary weather files in EPW folder.
Coercing year to 2005
Filtering dates
Saving file EPWs\metdata_temp.csv, # points: 8760
Calculating Sun position for Metdata that is right-labeled  with a delta of -30 mins. i.e. 12 is 11:30 sunpos
Loaded  EPWs\metdata_temp.csv
message: There were 448 sun up hours in this climate file
Total Ibh/Lbh: 0.000000
Created nespelem.oct
Linescan in process: nespelem_May_EMPTYSCAN_Front
Linescan in process: nespelem_May_EMPTYSCAN_Back
Saved: results\irr_nespelem_May_EMPTYSCAN.csv
Created nespelem.oct
Created nespelem.oct
Linescan in process: n

8760 line in WeatherFile. Assuming this is a standard hourly WeatherFile for the year for purposes of saving Gencumulativesky temporary weather files in EPW folder.
Coercing year to 2020
Filtering dates
Saving file EPWs\metdata_temp.csv, # points: 8760
Calculating Sun position for Metdata that is right-labeled  with a delta of -30 mins. i.e. 12 is 11:30 sunpos
Loaded  EPWs\metdata_temp.csv
message: There were 438 sun up hours in this climate file
Total Ibh/Lbh: 0.000000
Created nespelem.oct
Linescan in process: nespelem_Aug_EMPTYSCAN_Front
Linescan in process: nespelem_Aug_EMPTYSCAN_Back
Saved: results\irr_nespelem_Aug_EMPTYSCAN.csv
Created nespelem.oct
Created nespelem.oct
Linescan in process: nespelem_Aug_az_180_Module_Analysis_Front
Linescan in process: nespelem_Aug_az_180_Module_Analysis_Back
Saved: results\irr_nespelem_Aug_az_180_Module_Analysis_Front.csv
Saved: results\irr_nespelem_Aug_az_180_Module_Analysis_Back.csv
Linescan in process: nespelem_Aug_az_180_Ground_Analysis_Front


<Figure size 432x288 with 0 Axes>

## Plotting attempts. Code from below brought to man code

In [ ]:
#test = os.path.join(resultsfolder, 'irr_nespelem_May_az_180_Ground_Analysis_Front_test.csv')
#df = pd.read_csv(test)
#np.reshape(np.ravel(num_array, order='C'), (4,2), order='F')
#Wm = np.asarray(df['Wm2Front']) 
#Wms = np.reshape(np.ravel(Wm, order='C'), (10,15), order='F')
#((np.asarray(df['x'])).reshape(np.ravel(num_arrorder='C'), (15,10), order='F')
#print(Wms)
#fig = plt.imshow(Wms, cmap='hot', vmin=np.min(Wms).min(), vmax=np.max(Wms).max(), interpolation='none', aspect = 'equal')
#result = df.pivot(index='y',columns='x',values='Wm2Front')
#print(result)
#fig2 = plt.imshow(result, cmap='hot', vmin=np.min(result).min(), vmax=np.max(result).max(), interpolation='none', aspect = 'equal')
#plt.colorbar(label='G$_{Ground}$')
#plt.title('May Bifacial (az = 180), in matrix form')
#fig.axes.get_yaxis().set_visible(False)
#fig.axes.get_xaxis().set_visible(False)


In [ ]:
# Create Irradiance HeatMap for May
test2 = os.path.join(resultsfolder, 'irr_nespelem_May_az_180_Ground_Analysis_Front.csv')
df2 = pd.read_csv(test2)
#np.reshape(np.ravel(num_array, order='C'), (4,2), order='F')
Wm2 = np.asarray(df2['Wm2Front']) 
Wms2 = np.reshape(np.ravel(Wm2, order='C'), (122,217), order='F')
#((np.asarray(df['x'])).reshape(np.ravel(num_arrorder='C'), (15,10), order='F')
#print(Wms)
fig = plt.imshow(Wms2, cmap='hot', vmin=np.min(Wms2).min(), vmax=np.max(Wms2).max(), interpolation='none', aspect = 'equal')
#result = df.pivot(index='y',columns='x',values='Wm2Front')
#print(result)
#fig2 = plt.imshow(result, cmap='hot', vmin=np.min(result).min(), vmax=np.max(result).max(), interpolation='none', aspect = 'equal')
plt.colorbar(label='G$_{Ground}$')
plt.title('May Bifacial (az=180), in matrix form')
fig.axes.get_yaxis().set_visible(False)
fig.axes.get_xaxis().set_visible(False)
#plt.savefig('May_Bifacial_az_180.png')

In [ ]:
# For loop creates and saves heat map of irradiance factor for all 12 months and all 3 azimuths per month
for ti in range (0, len(starttimes)):
    starttime = pd.to_datetime(starttimes[ti])
    endtime = pd.to_datetime(endtimes[ti])
    month = months[ti]   
    azimuths_string = ['180','150','210']
    for az in range (0, len(azimuths_string)):
        #Run Build scene and run scans for all 3 azimuths
        azimuth = azimuths_string[az]
        #Read each azumith file for Wm2 Front
        file = os.path.join(resultsfolder, 'irr_nespelem_'+month+'_az_'+azimuth+'_Ground_Analysis_Front.csv')
        df2 = pd.read_csv(file)
        Wm2 = np.asarray(df2['Wm2Front']) 
        Wms2 = np.reshape(np.ravel(Wm2, order='C'), (122,217), order='F')
        #Read each empty file for Wm2Front
        empty_scan = os.path.join(resultsfolder, 'irr_nespelem_EMPTY_'+month+'_EMPTYSCAN.csv')
        de = pd.read_csv(empty_scan)
        Wmempty = np.asarray(de['Wm2Front']) 
        Wms_empty = np.reshape(np.ravel(Wmempty, order='C'), (122,217), order='F')
        #Calculate Irradiance Facor and plot and save heat map of each month_azumith
        plt.figure(4)
        shading = Wms2/Wms_empty
        fig = plt.imshow(shading, cmap='hot', vmin=np.min(shading).min(), vmax=np.max(shading).max(), interpolation='none', aspect = 'equal')
        plt.colorbar(label='Irr Factor')
        plt.title(month+'_Bifacial Irr Factor (az='+azimuth+'), in matrix form')
        fig.axes.get_yaxis().set_visible(False)
        fig.axes.get_xaxis().set_visible(False)
        plt.savefig(month+'_Bifacial_az_'+azimuth+'.png')
        plt.clf()
    

In [ ]:
#nespelem_ground_irrad_150 = data150g['Wm2Front'][0]
#nespelem_ground_irrad_180 = data180g['Wm2Front'][0]
#nespelem_ground_irrad_210 = data210g['Wm2Front'][0]
#print(nespelem_ground_irrad_150, nespelem_ground_irrad_180, nespelem_ground_irrad_210)

## 2. Calculate GHI for Comparisons

### Option : Raytrace of Empty Field for Irradiance Factor Calculations

In [ ]:
# Prepare to run for loop of start and end times for May to October
#starttimes =['2005-05-01 1:30:0', '2007-06-01 1:30:0', '2020-07-01 1:30:0', '2020-08-01 1:30:0', '2002-09-01 1:30:0', '1998-10-01 1:30:0']
#endtimes = ['2005-05-31 23:30:0', '2007-06-30 23:30:0','2020-07-31 23:30:0', '2020-08-31 23:30:0', '2002-09-30 23:30:0', '1998-10-31 23:30:0']
#months = ['May','Jun','Jul','Aug','Sep','Oct']
#demo = bifacial_radiance.RadianceObj(sim_general_name,str(testfolder))  
#demo.setGround(albedo)
demo.makeModule(name=moduletype, x=0.001, y=0.001, xgap = 0)
for ti in range (0, len(starttimes)):
    starttime = pd.to_datetime(starttimes[ti])
    endtime = pd.to_datetime(endtimes[ti])
    month = months[ti]
    demo.readWeatherFile(epwfile, starttime=starttime, endtime=endtime) # Name and path
    demo.genCumSky() 

    sim_name = sim_general_name+'_'+ month
    sceneDict = {'tilt':0,'pitch':2,'clearance_height':0.005,'azimuth':180, 'nMods': 1, 'nRows': 1} 
    scene = demo.makeScene(moduletype=moduletype,sceneDict=sceneDict, radname = sim_name)
    octfile = demo.makeOct(octname = demo.basename)  
    analysis = bifacial_radiance.AnalysisObj(octfile=octfile, name=sim_name)
    frontscan, backscan = analysis.moduleAnalysis(scene=scene,sensorsx = sensorsx_gb,sensorsy=sensorsy_gb)
    emptyscan = frontscan.copy() 
    emptyscan['xstart'] = xstart
    emptyscan['xinc'] = 0
    emptyscan['sx_xinc'] = xinc
    emptyscan['ystart'] = ystart
    emptyscan['yinc'] = yinc
    emptyscan['zstart'] = 0.05
    emptyscan['zinc'] = 0
    emptyscan['orient'] = '0 0 -1'
    emptybackscan = emptyscan.copy()
    emptybackscan['orient'] = '0 0 1'
    analysis.analysis(octfile, name=sim_name+'_EMPTYSCAN', frontscan=emptyscan, backscan=emptybackscan)


In [ ]:
empty_test = os.path.join(resultsfolder, 'irr_nespelem_EMPTY_May_EMPTYSCAN.csv')
de = pd.read_csv(empty_test)
#np.reshape(np.ravel(num_array, order='C'), (4,2), order='F')
Wmempty = np.asarray(de['Wm2Front']) 
Wms_empty = np.reshape(np.ravel(Wmempty, order='C'), (122,217), order='F')
#((np.asarray(df['x'])).reshape(np.ravel(num_arrorder='C'), (15,10), order='F')
#print(Wms)
plt.figure(1)
fig = plt.imshow(Wms_empty, cmap='hot', vmin=np.min(Wms_empty).min(), vmax=np.max(Wms_empty).max(), interpolation='none', aspect = 'equal')
#result = df.pivot(index='y',columns='x',values='Wm2Front')
#print(result)
#fig2 = plt.imshow(result, cmap='hot', vmin=np.min(result).min(), vmax=np.max(result).max(), interpolation='none', aspect = 'equal')
plt.colorbar(label='G$_{Ground}$')
plt.title('May Bifacial Empty (az=180), in matrix form')
fig.axes.get_yaxis().set_visible(False)
fig.axes.get_xaxis().set_visible(False)
#plt.savefig('May_Bifacial_az_180.png')

month = 'May'
plt.figure(2)
shading = Wms2/Wmse
print(shading)
fig = plt.imshow(shading, cmap='hot', vmin=np.min(shading).min(), vmax=np.max(shading).max(), interpolation='none', aspect = 'equal')
plt.colorbar(label='Irr Factor')
plt.title('May Bifacial Irr Factor (az=180), in matrix form')
fig.axes.get_yaxis().set_visible(False)
fig.axes.get_xaxis().set_visible(False)
#plt.savefig('May_Bifacial_az_180.png')



In [ ]:
#resname = os.path.join(resultsfolder, 'irr__EMPTYSCAN.csv')
#data = pd.read_csv(resname)
#nespelem_year_empty = data['Wm2Front'][0]
#print("Empty field YEARLY TOTAL Wh/m2:", nespelem_year_empty)

#shading = np.array([nespelem_ground_irrad_150/nespelem_year_empty,nespelem_ground_irrad_180/nespelem_year_empty,nespelem_ground_irrad_210/nespelem_year_empty])
#print("Percent Unshaded at 150, 180 and 210 degree azimuths",shading)

<a id='step5b'></a>

<a id='step5c'></a>

## 7. Plot

In [ ]:
tilts_l = list(trees['tilt'].unique())
ch_l = list(trees['clearance_height'].unique())
print(tilts_l)
print(ch_l)

In [ ]:
for tilt in tilts_l:
    for clearance_height in ch_l:
        df2=trees.loc[df['tilt']==tilts[1]]
        df3 = df2.loc[df2['clearance_height']==clearance_heights[2]]
        df3['pitch']=df3['pitch'].round(1)
        df3['xgap']=df3['xgap'].round(1)

        sns.set(font_scale=2) 
        table = df3.pivot('pitch', 'xgap', 'TreeIrrad_percent_GHI')
        ax = sns.heatmap(table, cmap='hot', vmin = 22, vmax= 35, annot=True)
        ax.invert_yaxis()
        figtitle = 'Clearance Height ' + str(clearance_height/ft2m)+'  ft, Tilt ' + str(tilt) + '$^\circ$'
        plt.title(figtitle)
        print(table)
        plt.show()
        
